In [ ]:
# Import the required libraries
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.


url = 'https://www.funda.nl/detail/koop/verkocht/amsterdam/appartement-amsteldijk-143-3/43425080/'

In [2]:
# Make a get request to retrieve page
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})

# Resolve the response
if response.status_code==200:
    html_content = response.text.encode('utf-8')
else:
    print(response.status_code)


# Parse HTML content
# Parse HTML page with BeautifulSoup
soup = BeautifulSoup(html_content)

In [26]:
headless = soup.find_all('div',   {"id" :  re.compile('headlessui-disclosure-panel.*')})

In [5]:
from openai import OpenAI
import os

print(os.environ.items())
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


ItemsView(environ({'CARGO_HOME': '/Users/christoph.kieslich/.local/share/mise/installs/rust/1.76.0', 'CHAT_ID': '-1002016110840', 'CM_computer_name': 'chrisikea', 'CM_hostname': 'chrisdoc', 'COLORTERM': 'truecolor', 'COMMAND_MODE': 'unix2003', 'EDITOR': 'nvim', 'ELECTRON_NO_ATTACH_CONSOLE': '1', 'FUNDA_DB': 'funda.db', 'GOPATH': '/Users/christoph.kieslich/.local/share/mise/installs/go/1.22.1/packages', 'GOROOT': '/Users/christoph.kieslich/.local/share/mise/installs/go/1.22.1/go', 'HOME': '/Users/christoph.kieslich', 'HOMEBREW_CELLAR': '/opt/homebrew/Cellar', 'HOMEBREW_PREFIX': '/opt/homebrew', 'HOMEBREW_REPOSITORY': '/opt/homebrew', 'INFOPATH': '/opt/homebrew/share/info:/opt/homebrew/share/info:', 'JAVA_HOME': '/Users/christoph.kieslich/.local/share/mise/installs/java/openjdk-21', 'LANG': 'en_US.UTF-8', 'LIBGL_ALWAYS_SOFTWARE': 'true', 'LOGNAME': 'christoph.kieslich', 'LaunchInstanceID': 'FD336D70-B7C7-476C-8FEA-0BFEE2B2357E', 'MANPATH': '/opt/homebrew/share/man:/opt/homebrew/share/man

In [75]:
content="\n".join(list(map(lambda x: x.get_text(), headless)))
prompt = f"""
You are given the information about a dutch house listing and your task is to generate a summary of the listing.

Please format the output in Markdownv2 so that it can be used to send a message from a telegram bot to a house hunter. Make sure that there is a tl;dr execute summary at the beginning with the most important information like size, number of bedrooms, highlights and location! Only include the most important information and discard everything else. The message should be maximum 100 words long with the most important information at the beginning.

Do not wrap the text in a markdown code block, just use the markdown syntax for formatting.

Content:
\"\"\"
{content}
\"\"\"
"""

completion=client.chat.completions.create(messages=[{"role":"user","content":prompt}], model="gpt-4o")

In [90]:
message=completion.choices[0].message.content
print(message)

**TL;DR:**
**Location:** Amsteldijk 143 3, Amsterdam  
**Size:** 82 m²  
**Bedrooms:** 3  
**Highlights:** Luxe renovatie, dakterras met buitenkeuken, vloerverwarming, hoge plafonds, zonnig balkon, uitzicht op de Amstel.  

---

**Te Koop:** Compleet gerenoveerd appartement van ca. 82 m² in een goed onderhouden pand aan de Amstel. Dit high-end appartement beschikt over 3 slaapkamers, een luxe badkamer, een zonnig balkon, en een riant dakterras (33 m²) met buitenkeuken en buitenbad. Uitgerust met vloerverwarming en hoogwaardige afwerking. Perfect gelegen met gemak tot winkels, restaurants, parken, en openbaar vervoer. VvE servicekosten: €120/maand. Snelle oplevering mogelijk.


In [93]:
from telegram_bot import TelegramBot
import asyncio
telegram_token = os.environ["TELEGRAM_TOKEN"]
chat_id = os.environ["CHAT_ID"]
telegramBot = TelegramBot(chat_id, telegram_token)
import nest_asyncio
nest_asyncio.apply()

c = message.replace("**","*").replace(".", "\\.").replace(">", "\\>").replace("<", "\\<").replace("_","\\_").replace("-","\\-").replace("(","\\(").replace(")","\\)").replace("k\\.k\\.", "")
print(c)
telegramBot.postMessage(c)


*TL;DR:*
*Location:* Amsteldijk 143 3, Amsterdam  
*Size:* 82 m²  
*Bedrooms:* 3  
*Highlights:* Luxe renovatie, dakterras met buitenkeuken, vloerverwarming, hoge plafonds, zonnig balkon, uitzicht op de Amstel\.  

\-\-\-

*Te Koop:* Compleet gerenoveerd appartement van ca\. 82 m² in een goed onderhouden pand aan de Amstel\. Dit high\-end appartement beschikt over 3 slaapkamers, een luxe badkamer, een zonnig balkon, en een riant dakterras \(33 m²\) met buitenkeuken en buitenbad\. Uitgerust met vloerverwarming en hoogwaardige afwerking\. Perfect gelegen met gemak tot winkels, restaurants, parken, en openbaar vervoer\. VvE servicekosten: €120/maand\. Snelle oplevering mogelijk\.
*bold text*

_italic text_

[inline URL](http://www.example.com/)

`inline fixed-width code`
